# Design attention head

In [1]:
import torch
from torch import nn
from model import HRSeg, SelfAttention
import torch.nn.functional as F
from utils import count_parameters

c:\Users\dranh\anaconda3\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
x1 = torch.rand(1, 64, 72, 72)
x2 = torch.rand(1, 128, 36, 36)
x3 = torch.rand(1, 320, 18, 18)
x4 = torch.rand(1, 512, 9, 9)

In [10]:
class AttentionHead(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.upConv4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=64, kernel_size=4, stride=4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=2, stride=2),
            nn.ReLU(),
            SelfAttention(64)
        ) # [1, 64, 72, 72]
        
        self.upConv3 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=320, out_channels=64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=2, stride=2),
            nn.ReLU(),
            SelfAttention(64)
        ) # [1, 64, 72, 72]

        self.upConv2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2),
            nn.ReLU(),
            SelfAttention(64)
        ) # [1, 64, 72, 72]

        self.upConv_out = nn.ConvTranspose2d(in_channels=64, out_channels=1, kernel_size=4, stride=4)

    def forward(self, x1, x2, x3, x4):
        x = self.upConv4(x4)
        return x

In [16]:
model = HRSeg()

Loaded state dict for encoder: pretrained_pth/mit_b2.pth


In [23]:
encoded = model.encoder(torch.rand(1, 3, 288, 288))
att_map = model.att_head(*encoded)
att_map.shape

torch.Size([1, 1, 288, 288])

# Test DF

In [1]:
import pandas as pd
from statistic import Comparator

comparator = Comparator("HRSeg8", "ssformer_S")

comparator.df

Processing  HRSeg8
Loaded calc result of HRSeg8 from cache
Processing  ssformer_S
Loaded calc result of ssformer_S from cache


,name,ds_name,img_idx,dice,size
0,HRSeg8,CVC-300,0,95.393233,9.628920
1,HRSeg8,CVC-300,1,92.616489,7.178049
2,HRSeg8,CVC-300,2,91.567193,5.087805
3,HRSeg8,CVC-300,3,90.247782,4.234843
4,HRSeg8,CVC-300,4,66.564152,4.618467
...,...,...,...,...,...
793,ssformer_s,ETIS-LaribPolypDB,191,0.063091,0.133857
794,ssformer_s,ETIS-LaribPolypDB,192,0.043346,0.112139
795,ssformer_s,ETIS-LaribPolypDB,193,0.002775,0.196561
796,ssformer_s,ETIS-LaribPolypDB,194,0.040048,0.113153


In [49]:
df = comparator.df

In [51]:
df[df['name'] == 'HRSeg8']

,name,ds_name,img_idx,dice,size
0,HRSeg8,CVC-300,0,95.393233,9.628920
1,HRSeg8,CVC-300,1,92.616489,7.178049
2,HRSeg8,CVC-300,2,91.567193,5.087805
3,HRSeg8,CVC-300,3,90.247782,4.234843
4,HRSeg8,CVC-300,4,66.564152,4.618467
...,...,...,...,...,...
793,HRSeg8,ETIS-LaribPolypDB,191,0.134235,0.133857
794,HRSeg8,ETIS-LaribPolypDB,192,6.310312,0.112139
795,HRSeg8,ETIS-LaribPolypDB,193,0.366378,0.196561
796,HRSeg8,ETIS-LaribPolypDB,194,2.720565,0.113153


In [70]:
import numpy as np
mean_dice_by_interval = df.groupby(pd.cut(df['size'], np.arange(0, 100, 5))).mean().dropna()['dice']
mean_dice_by_interval

size
(0, 5]      80.727775
(5, 10]     89.261294
(10, 15]    89.212892
(15, 20]    89.673092
(20, 25]    86.419237
(25, 30]    71.753589
(30, 35]    58.921206
(35, 40]    64.485846
(40, 45]    67.471639
(45, 50]    45.809396
(50, 55]    30.850380
(55, 60]    92.267263
(60, 65]    90.534504
Name: dice, dtype: float64

In [ ]:
df.groupby(pd.cut(df['size'], np.arange(0, 100, 5))).mean().dropna()['dice']

In [68]:
mean_dice_by_interval.index.astype(str).to_list()

['(0, 5]',
 '(5, 10]',
 '(10, 15]',
 '(15, 20]',
 '(20, 25]',
 '(25, 30]',
 '(30, 35]',
 '(35, 40]',
 '(40, 45]',
 '(45, 50]',
 '(50, 55]',
 '(55, 60]',
 '(60, 65]']

In [69]:
series.to_list()

[80.72777532348252,
 89.26129377944756,
 89.21289205599506,
 89.67309191287939,
 86.41923739594327,
 71.75358948641463,
 58.92120643205692,
 64.48584596976832,
 67.47163940900255,
 45.80939647865539,
 30.850379968536807,
 92.26726305630842,
 90.5345042302356]